#GCP Setup and Imports

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

NAME           PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
index-creator  GCE       4                                       RUNNING  us-central1-b


In [20]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*


-rw-r--r-- 1 root root 247882 Jan  8 20:06 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [21]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [22]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")

sys.path.insert(0,SparkFiles.getRootDirectory())

from inverted_index_gcp import InvertedIndex

22/01/08 20:46:32 WARN org.apache.spark.SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '313245508_bucket' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh
postings_gcp/0_000.bin
postings_gcp/0_001.bin
postings_gcp/0_002.bin
postings_gcp/0_003.bin
postings_gcp/0_004.bin
postings_gcp/0_005.bin
postings_gcp/0_006.bin
postings_gcp/0_007.bin
postings_gcp/0_008.bin
postings_gcp/0_009.bin
postings_gcp/0_010.bin
postings_gcp/0_011.bin
postings_gcp/0_012.bin
postings_gcp/0_013.bin
postings_gcp/0_014.bin
postings_gcp/0_015.bin
postings_gcp/0_016.bin
postings_gcp/0_017.bin
postings_gcp/0_018.bin
postings_gcp/0_019.bin
postings_gcp/0_020.bin
postings_gcp/0_021.bin
postings_gcp/0_022.bin
postings_gcp/0_023.bin
postings_gcp/0_024.bin
postings_gcp/0_025.bin
postings_gcp/0_026.bin
postings_gcp/0_027.bin
postings_gcp/0_028.bin
postings_gcp/0_posting_locs.pickle
postings_gcp/100_000.bin
postings_gcp/100_001.bin
postings_gcp/100_002.bin
postings_gcp/100_003.bin
postings_gcp/100_004.bin
postings_gcp/100_005.bin
postings_gcp/100_006.bin
postings_gcp/100_007.bin
postings_gcp/100_008.bin
postings_gcp/100_009.bin
postings_gcp/100_010.bin
postings

#Functions declarations

In [8]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(text, id, rmv_stopwords=True):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  result = []
  if rmv_stopwords:
    counter = Counter([i for i in tokens if i not in all_stopwords])
  else:
    counter = Counter([i for i in tokens])
    
  for key,val in counter.items():
    tup_to_append = (key, (id, val))
    result.append(tup_to_append)
    
  return result


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl)


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  result = postings.map(lambda x : (x[0], len(x[1])))
  return result



NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings, folder_name):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  result_before_grouping = postings.map(lambda x : (token2bucket_id(x[0]), x))
  result_after_grouping = result_before_grouping.groupByKey()
  result_after_grouping = result_after_grouping.map(lambda x : InvertedIndex.write_a_posting_list(x, bucket_name, folder_name))
  return result_after_grouping
  



  TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

#Inverted Index - Body

In [ ]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
doc_text_pairs = parquetFile.select("text", "id").rdd

# Count number of wiki pages
parquetFile.count()


# time the index creation time
t_start = time()

# word counts map
word_counts_text = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1], True))
postings = word_counts_text.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered_text = postings.filter(lambda x: len(x[1])>50)
text_df = calculate_df(postings_filtered_text)
w2df_dict_text = text_df.collectAsMap()
# partition posting lists and write out
posting_locs_list_text = partition_postings_and_write(postings_filtered_text, 'body').collect()

# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_body'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

index_const_time = time() - t_start


# Create inverted index instance for text
inverted_text = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_text.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted_text.df = w2df_dict_text
# write the global stats out
inverted_text.write_index('.', 'body_index')
# upload to gs
index_src = "body_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_body/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://body_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 18.4 MiB/ 18.4 MiB]                                                
Operation completed over 1 objects/18.4 MiB.                                     


#Inverted Index - Title

In [18]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
doc_text_pairs = parquetFile.select("title", "id").rdd

# Count number of wiki pages
parquetFile.count()


# time the index creation time
t_start = time()

# word counts map
word_counts_title = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1], True))
postings = word_counts_title.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered_title = postings.filter(lambda x: len(x[1])>50)
text_df = calculate_df(postings_filtered_title)
w2df_dict_text = text_df.collectAsMap()
# partition posting lists and write out
posting_locs_list_text = partition_postings_and_write(postings_filtered_title, 'title').collect()

# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_title'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

index_const_time = time() - t_start


# Create inverted index instance for text
inverted_text = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_text.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted_text.df = w2df_dict_text
# write the global stats out
inverted_text.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_title/{index_src}'
!gsutil cp $index_src $index_dst

AttributeError: 'SparkContext' object has no attribute 'set'

#Inverted Index - Anchor

In [19]:
def split_text_anchor(list_anchor):
  if list_anchor is not None:
      for row in list_anchor:
        if row is not None:
            lst = []
            print(row.text)
            split_text = row.text.split(" ")
            for term in split_text:
                if term is not None:
                    lst.append(((term,row.id),1))
        return lst
    


full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
doc_anchor_text_pairs = parquetFile.select("anchor_text", "id").rdd

# Count number of wiki pages
parquetFile.count()

dict_of_anchor_words = {}


# time the index creation time
t_start = time()

#return a term and docID tuple for terms that are pointing to a given docid
term_docID_filter = doc_anchor_text_pairs.filter(lambda x: x.anchor_text is not None and len(x.anchor_text)!=0)
term_docID = term_docID_filter.flatMap(lambda x: split_text_anchor(x.anchor_text))

# term_and_docID_tf = doc_anchor_text_pairs.flatMap(lambda x: anchor_split(x.anchor_text))
term_docID_and_tf = term_docID.map(lambda x : (x[0][0],(x[0][1],x[1])))
postings = term_docID_and_tf.groupByKey().mapValues(list)
sorted_postings = postings.map(lambda x: (x[0],sorted(x[1],key = lambda y:y[1],reverse=True)))


w2df = calculate_df(sorted_postings)
w2df_dict_anchor = w2df.collectAsMap()
posting_locs_list = partition_postings_and_write(sorted_postings, 'anchor').collect()

# # Code for normal corpus - will hold dict of {word: (doc_id, 1), (doc_id, 1)}
# for row in doc_text_pairs.collectAsMap():
#   for pair in row.anchor_text:
#     for word in pair.text.split(' '):
#       word = word.lower()
      
#       if word in all_stopwords:
#         continue

#       if word not in dict_of_anchor_words.keys():
#         dict_of_anchor_words[word] = Counter()
#         dict_of_anchor_words[word][pair.id] = 1
      
#       else:
#         dict_of_anchor_words[word][pair.id] += 1
        
# posting_anchor_text = list(map(lambda x: (x[0], [(int(i[0]), i[1]) for i in x[1].most_common()]), list(dict_of_anchor_words.items())))
# posting_anchor_text = spark.sparkContext.parallelize(posting_anchor_text)


# anchor_text_df = calculate_df(posting_anchor_text)
# w2df_dict_anchor_text = anchor_text_df.collectAsMap()
# posting_locs_list_anchor_text = partition_postings_and_write(posting_anchor_text).collect()



# # word counts map
# word_counts_anchor = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1], True))
# postings = word_counts_anchor.groupByKey().mapValues(reduce_word_counts)
# # filtering postings and calculate df
# postings_filtered_anchor = postings.filter(lambda x: len(x[1])>50)
# text_df = calculate_df(postings_filtered_anchor)
# w2df_dict_text = text_df.collectAsMap()
# # partition posting lists and write out
# posting_locs_list_text = partition_postings_and_write(postings_filtered_anchor, 'anchor').collect()

# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_anchor'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

index_const_time = time() - t_start


# Create inverted index instance for text
inverted_text = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted_text.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted_text.df = w2df_dict_anchor
# write the global stats out
inverted_text.write_index('.', 'anchor_index')
# upload to gs
index_src = "anchor_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_anchor/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://anchor_index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 16.7 MiB/ 16.7 MiB]                                                
Operation completed over 1 objects/16.7 MiB.                                     


#Page View

In [ ]:
import numpy as np
import bz2
from functools import partial
from collections import Counter
import pickle
from itertools import islice
from xml.etree import ElementTree
import codecs
import csv
import time
import os
import re
from pathlib import Path


# Paths
# Using user page views (as opposed to spiders and automated traffic) for the 
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path) 
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB) 
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly 
# total number of page views (5). Then, remove lines with article id or page 
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same 
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)